# 作業 : 實作英文-德文翻譯機器人
***
## [作業目標]

用 pytorch 實作一個英文-德文翻譯機器人

## [作業目標]

*   語言資料處理
*   使用 LSTM 建構 Encoder: EncoderLSTM
*   使用 LSTM 建構 Decoder: DecoderLSTM
*   搭建 Sequence to Sequence 模型: Seq2Seq
*   撰寫訓練函式
*   撰寫測試函式

## [問題]

在 Colab 實際上執行完這個範例後，請改用 BiLSTM 來建構 Encoder 與 Decoder


## 安裝 spacy

We'll also make use of spaCy to tokenize our data. To install spaCy, follow the instructions here making sure to install both the English and German models with:

In [3]:
! nvidia-smi

Sun May 23 06:28:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
!pip uninstall spacy -y
!pip install -U spacy

Uninstalling spacy-2.2.4:
  Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 12.8MB 241kB/s 
     |████████████████████████████████| 9.1MB 172kB/s 
     |████████████████████████████████| 460kB 48.6MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 1.1MB 49.3MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
     |████████████████████████████████| 122kB 63.2MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=567ab676965f612797eb211676558a655ccf7ba4d01d6463543c0c964b3ce563
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Foun

## 引用需要的模組

In [6]:
import jieba
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import numpy as np
import pandas as pd
import spacy
import random
from torchtext.data.metrics import bleu_score
from pprint import pprint
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

## 下載英文預料

In [7]:
!mkdir ./data
!mkdir ./data/multi30k
!python -m spacy download en
!ls ./data/multi30k -al
spacy_english = spacy.load("en_core_web_sm")
!ls ./data/multi30k -al

2021-05-23 06:30:35.301346: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.7MB 224kB/s 
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
total 8
drwxr-xr-x 2 root root 4096 May 23 06:30 .
drwxr-xr-x 3 root root 4096 May 23 06:30 ..
total 8
drwxr-xr-x 2 root root 4096 May 23 06:30 .
drwxr-xr-x 3 root root 4096 May 23 06:30 ..


## 下載德語語料

In [8]:
!python -m spacy download de
spacy_de = spacy.load("de_core_news_sm")
!ls ./data/multi30k -al

2021-05-23 06:30:43.731647: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     |████████████████████████████████| 19.3MB 156kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
total 8
drwxr-xr-x 2 root root 4096 May 23 06:30 .
drwxr-xr-x 3 root root 4096 May 23 06:30 ..


In [9]:
def tokenize_de(text):
  return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_english(text):
  return [token.text for token in spacy_english.tokenizer(text)]

### Sample Run ###

sample_text = "I love machine learning"
print(tokenize_english(sample_text))

german = Field(tokenize=tokenize_de, lower=True,
               init_token="<sos>", eos_token="<eos>")

english = Field(tokenize=tokenize_english, lower=True,
               init_token="<sos>", eos_token="<eos>")

#translate sentence from english to german
train_data, valid_data, test_data = Multi30k.splits(exts = (".en", ".de"),
                                                    fields=(english, german))

english.build_vocab(train_data, max_size=10000, min_freq=3)
german.build_vocab(train_data, max_size=10000, min_freq=3)

print(f"Unique tokens in target (en) vocabulary: {len(english.vocab)}")
print(f"Unique tokens in source (german) vocabulary: {len(german.vocab)}")

['I', 'love', 'machine', 'learning']
downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 1.62MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 275kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 267kB/s]


Unique tokens in target (en) vocabulary: 4556
Unique tokens in source (german) vocabulary: 5374


In [10]:
# word_2_idx = dict(e[2])
# idx_2_word = {}
# for k,v in word_2_idx.items():
#   idx_2_word[v] = k

In [11]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

print(train_data[5].__dict__.keys())
pprint(train_data[5].__dict__.values())

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
dict_keys(['src', 'trg'])
dict_values([['a', 'man', 'in', 'green', 'holds', 'a', 'guitar', 'while', 'the', 'other', 'man', 'observes', 'his', 'shirt', '.'], ['ein', 'mann', 'in', 'grün', 'hält', 'eine', 'gitarre', ',', 'während', 'der', 'andere', 'mann', 'sein', 'hemd', 'ansieht', '.']])


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_data, valid_data, test_data), 
                                                                      batch_size = BATCH_SIZE, 
                                                                      sort_within_batch=True,
                                                                      sort_key=lambda x: len(x.src),
                                                                      device = device)

In [13]:
count = 0
max_len_eng = []
max_len_ger = []
for data in train_data:
  max_len_ger.append(len(data.src))
  max_len_eng.append(len(data.trg))
  if count < 10 :
    print("English - ",*data.src, " Length - ", len(data.src))
    print("German - ",*data.trg, " Length - ", len(data.trg))
    print()
  count += 1

print("Maximum Length of English sentence {} and German sentence {} in the dataset".format(max(max_len_eng),max(max_len_ger)))
print("Minimum Length of English sentence {} and German sentence {} in the dataset".format(min(max_len_eng),min(max_len_ger)))

English -  two young , white males are outside near many bushes .  Length -  11
German -  zwei junge weiße männer sind im freien in der nähe vieler büsche .  Length -  13

English -  several men in hard hats are operating a giant pulley system .  Length -  12
German -  mehrere männer mit schutzhelmen bedienen ein antriebsradsystem .  Length -  8

English -  a little girl climbing into a wooden playhouse .  Length -  9
German -  ein kleines mädchen klettert in ein spielhaus aus holz .  Length -  10

English -  a man in a blue shirt is standing on a ladder cleaning a window .  Length -  15
German -  ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster .  Length -  15

English -  two men are at the stove preparing food .  Length -  9
German -  zwei männer stehen am herd und bereiten essen zu .  Length -  10

English -  a man in green holds a guitar while the other man observes his shirt .  Length -  15
German -  ein mann in grün hält eine gitarre , während der andere

In [14]:
count = 0
for data in train_iterator:
  if count < 1 :
    print("Shapes", data.src.shape, data.trg.shape)
    print()
    print("English - ",*data.src, " Length - ", len(data.src))
    print()
    print("German - ",*data.trg, " Length - ", len(data.trg))
    temp_ger = data.src
    temp_eng = data.trg
    count += 1

Shapes torch.Size([15, 32]) torch.Size([22, 32])

English -  tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2], device='cuda:0') tensor([   4,   21,    7,    4,    4,    4,   16,  113,    4,    4,    4,    4,
           4,    4,    7,    4,    4,    4,    4,    4,    7,    4,    4,    4,
           4,    4,    4,   13,    4,    4, 2691,    4], device='cuda:0') tensor([  35,  233,  348,  178,   14,  120,   30,  326,    9,   33,   38,   39,
         224,   24,   34,   14,   55,    9,   53,    9,   33,   38,  183,   64,
         120,    9,   55, 1890,    9,    9,  304,   35], device='cuda:0') tensor([1027,    9,   10,   42,    6,    6,   17,   22,   11,    6,   12,  406,
         105,    9,  189, 2617,   10,    6,   34,   11,    6,   12,   13,    6,
          22,   10,    6, 2017,   10,   11,    4,   89], device='cuda:0') tensor([   8,   10,    6,  210,  148,   21,   36,   62,    4,    4,   19,   13,
         558,    6,   10,  

In [15]:
temp_eng_idx = (temp_eng).cpu().detach().numpy()
temp_ger_idx = (temp_ger).cpu().detach().numpy()

In [16]:
df_eng_idx = pd.DataFrame(data = temp_eng_idx, columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_idx.index.name = 'Time Steps'
df_eng_idx.index = df_eng_idx.index + 1 
# df_eng_idx.to_csv('/content/idx.csv')
df_eng_idx


,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
2,5,5,5,5,8,8,18,76,5,5,8,8,5,5,15,8,5,5,5,5,39,8,8,8,8,5,5,11,5,5,0,5
3,32,130,49,3183,16,16,30,330,13,25,36,638,550,96,26,16,49,13,70,13,25,113,709,67,165,13,49,0,13,13,3220,32
4,156,13,48,25,7,7,52,11,10,7,22,9,844,13,83,60,61,7,26,10,20,9,67,11,11,7,7,347,108,10,23,29
5,12,48,20,29,313,6,12,102,8,6,41,21,193,7,7,21,19,6,217,8,3813,17,11,6,380,15,555,1584,21,18,21,55
6,24,147,63,27,198,405,24,827,16,257,12,15,994,14,203,14,0,50,12,16,475,23,226,1463,250,97,12,43,6,45,6,91
7,317,9,10,14,9,128,455,10,489,79,6,253,56,153,219,78,10,40,6,11,69,444,477,9,542,44,6,0,0,9,487,11
8,47,14,37,2676,17,68,44,1572,822,37,1801,9,9,89,10,3412,238,31,433,524,42,27,10,21,1882,544,0,196,27,52,200,6
9,6,628,23,2161,8,12,382,167,12,23,7,349,35,303,11,44,7,47,12,47,33,6,201,24,12,10,7,0,6,7,9,78


In [17]:
idx_2_word = {idx: word for idx, word in enumerate(english.vocab.itos)}
df_eng_word = pd.DataFrame(columns = [str("S_")+str(x) for x in np.arange(1, 33)])
df_eng_word = df_eng_idx.replace(idx_2_word)
# df_eng_word.to_csv('/content/Words.csv')
df_eng_word

,S_1,S_2,S_3,S_4,S_5,S_6,S_7,S_8,S_9,S_10,S_11,S_12,S_13,S_14,S_15,S_16,S_17,S_18,S_19,S_20,S_21,S_22,S_23,S_24,S_25,S_26,S_27,S_28,S_29,S_30,S_31,S_32
Time Steps,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>,<sos>
2,.,.,.,.,on,on,to,over,.,.,on,on,.,.,",",on,.,.,.,.,street,on,on,on,on,.,.,and,.,.,<unk>,.
3,sitting,working,by,shack,two,two,men,dance,with,white,standing,match,rocky,grass,black,two,by,with,small,with,white,several,urban,hat,taking,with,by,<unk>,with,with,suburban,sitting
4,side,with,three,white,the,the,green,and,is,the,wearing,man,splashing,with,around,through,brown,the,black,is,at,man,hat,and,and,the,the,board,wall,is,shirt,blue
5,of,three,at,blue,she,in,of,'s,on,in,walking,an,coat,the,the,an,people,in,sunglasses,on,12,are,and,in,does,",",jackets,raising,an,to,an,child
6,young,pants,children,his,horse,both,young,cold,two,getting,of,",",kayak,woman,mouth,woman,<unk>,women,of,two,pulling,shirt,during,approaching,trees,hair,of,front,in,holding,in,sits
7,sweater,man,is,woman,man,play,reads,is,showing,running,in,outdoor,looking,baby,clothing,riding,is,down,in,and,into,dock,appears,man,eyes,her,in,<unk>,<unk>,man,instrument,and
8,water,woman,playing,broad,are,ball,her,observing,sleeps,playing,newly,man,man,stands,is,exposed,train,red,aged,gentleman,-,his,is,an,drawn,hit,<unk>,kids,his,green,band,in
9,in,helmets,shirt,interesting,on,of,machine,each,of,shirt,the,phone,dog,body,and,her,the,water,of,water,girl,in,watch,young,of,is,the,<unk>,in,the,man,riding


## 用 LSTM 搭建的 Encoder 類別: EncoderLSTM



In [18]:
class EncoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
    super(EncoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Regularization parameter
    self.dropout = nn.Dropout(p)
    self.tag = True

    # Shape --------------------> (4556 , 300) [input size(english vocab), embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)
    
    # Shape -----------> (300, 2, 1024) [embedding dims, hidden size, num layers]
    # self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)
    self.biLSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p, bidirectional=True)

  # Shape of x (26, 32) [Sequence_length, batch_size]
  def forward(self, x):

    # Shape -----------> (26, 32, 300) [Sequence_length , batch_size , embedding dims]
    embedding = self.dropout(self.embedding(x))
    
    # Shape --> outputs (26, 32, 1024) [Sequence_length , batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024) , (2, 32, 1024) [num_layers, batch_size size, hidden_size]
    # outputs, (hidden_state, cell_state) = self.LSTM(embedding)
    outputs, (hidden_state, cell_state) = self.biLSTM(embedding)
    hidden_state = hidden_state.view(self.num_layers, -1, self.hidden_size * 2) #use biLSTM --> hidden size *2
    cell_state = cell_state.view(self.num_layers, -1, self.hidden_size * 2) #use biLSTM --> hidden size *2

    return hidden_state, cell_state

input_size_encoder = len(english.vocab)
encoder_embedding_size = 300
hidden_size = 1024
num_layers = 2
encoder_dropout = 0.5

encoder_lstm = EncoderLSTM(input_size_encoder, encoder_embedding_size,
                           hidden_size, num_layers, encoder_dropout).to(device)
print(encoder_lstm)

EncoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(4556, 300)
  (biLSTM): LSTM(300, 1024, num_layers=2, dropout=0.5, bidirectional=True)
)


## 用 LSTM 搭建的 decoder 類別: DecoderLSTM


In [19]:
class DecoderLSTM(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, num_layers, p, output_size):
    super(DecoderLSTM, self).__init__()

    # Size of the one hot vectors that will be the input to the encoder
    #self.input_size = input_size

    # Output size of the word embedding NN
    #self.embedding_size = embedding_size

    # Dimension of the NN's inside the lstm cell/ (hs,cs)'s dimension.
    self.hidden_size = hidden_size

    # Number of layers in the lstm
    self.num_layers = num_layers

    # Size of the one hot vectors that will be the output from the decoder (german Vocab Size)
    self.output_size = output_size

    # Regularization parameter
    self.dropout = nn.Dropout(p)

    # Shape --------------------> (4556, 300) [input size(english vocab size), embedding dims]
    self.embedding = nn.Embedding(input_size, embedding_size)

    # Shape -----------> (300, 2, 1024*2) [embedding dims, hidden size, num layers]
    self.LSTM = nn.LSTM(embedding_size, hidden_size, num_layers, dropout = p)

    # Shape -----------> (1024*2, 5374) [hidden size, output size(german vocab size)]
    self.fc = nn.Linear(hidden_size, output_size)

  # Shape of x (32) [batch_size]
  def forward(self, x, hidden_state, cell_state):

    # Shape of x (1, 32) [1, batch_size]
    x = x.unsqueeze(0)

    # Shape -----------> (1, 32, 300) [1, batch_size, embedding dims]
    embedding = self.dropout(self.embedding(x))

    # Shape --> outputs (1, 32, 1024*2) [1, batch_size , hidden_size]
    # Shape --> (hs, cs) (2, 32, 1024*2) , (2, 32, 1024*2) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    outputs, (hidden_state, cell_state) = self.LSTM(embedding, (hidden_state, cell_state))

    # Shape --> predictions (1, 32, 5374) [ 1, batch_size , output_size(german vocab size)]
    predictions = self.fc(outputs)

    # Shape --> predictions (32, 5374) [batch_size , output_size(german vocab size)]
    predictions = predictions.squeeze(0)

    return predictions, hidden_state, cell_state

input_size_decoder = len(german.vocab)
decoder_embedding_size = 300
hidden_size = 1024 * 2
num_layers = 2
decoder_dropout = 0.5
output_size = len(german.vocab)

decoder_lstm = DecoderLSTM(input_size_decoder, decoder_embedding_size,
                           hidden_size, num_layers, decoder_dropout, output_size).to(device)
print(decoder_lstm)

DecoderLSTM(
  (dropout): Dropout(p=0.5, inplace=False)
  (embedding): Embedding(5374, 300)
  (LSTM): LSTM(300, 2048, num_layers=2, dropout=0.5)
  (fc): Linear(in_features=2048, out_features=5374, bias=True)
)


In [20]:
for batch in train_iterator:
  print(batch.src.shape)
  print(batch.trg.shape)
  break

x = batch.trg[1]
print(x)

torch.Size([20, 32])
torch.Size([23, 32])
tensor([  8,   5,   5,  76,  15,   5,  76,   5,   5,  76,   5,   8,   8,   8,
         99,   8,   5,  18,   8,  43,   5,   8, 389,   7,   5,  18,  14,   5,
          5,   5,   5,   5], device='cuda:0')


# Sequence to Sequence 類別

In [21]:
class Seq2Seq(nn.Module):
  def __init__(self, Encoder_LSTM, Decoder_LSTM):
    super(Seq2Seq, self).__init__()
    self.Encoder_LSTM = Encoder_LSTM
    self.Decoder_LSTM = Decoder_LSTM

  def forward(self, source, target, tfr=0.5):
    # Shape - Source : (10, 32) [(Sentence length english + some padding), Number of Sentences]
    batch_size = source.shape[1]

    # Shape - Source : (14, 32) [(Sentence length german + some padding), Number of Sentences]
    target_len = target.shape[0]
    target_vocab_size = len(german.vocab)
    
    # Shape --> outputs (14, 32, 5374) 5374:german vocab size
    outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

    # Shape --> (hs, cs) (2, 32, 1024) ,(2, 32, 1024) [num_layers, batch_size size, hidden_size] (contains encoder's hs, cs - context vectors)
    hidden_state, cell_state = self.Encoder_LSTM(source)

    # Shape of x (32 elements)
    x = target[0] # Trigger token <SOS>

    for i in range(1, target_len):
      # Shape --> output (32, 5374) 5374:german vocab size
      output, hidden_state, cell_state = self.Decoder_LSTM(x, hidden_state, cell_state)
      outputs[i] = output
      best_guess = output.argmax(1) # 0th dimension is batch size, 1st dimension is word embedding
      x = target[i] if random.random() < tfr else best_guess # Either pass the next word correctly from the dataset or use the earlier predicted word

    # Shape --> outputs (14, 32, 5374) 
    return outputs


In [22]:
# Hyperparameters

learning_rate = 0.001
writer = SummaryWriter(f"runs/loss_plot")
step = 0

model = Seq2Seq(encoder_lstm, decoder_lstm).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [23]:
model

Seq2Seq(
  (Encoder_LSTM): EncoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(4556, 300)
    (biLSTM): LSTM(300, 1024, num_layers=2, dropout=0.5, bidirectional=True)
  )
  (Decoder_LSTM): DecoderLSTM(
    (dropout): Dropout(p=0.5, inplace=False)
    (embedding): Embedding(5374, 300)
    (LSTM): LSTM(300, 2048, num_layers=2, dropout=0.5)
    (fc): Linear(in_features=2048, out_features=5374, bias=True)
  )
)

In [24]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
    spacy_eng = spacy.load("en_core_web_sm")

    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_eng(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    tokens.insert(0, english.init_token)
    tokens.append(english.eos_token)
    text_to_indices = [english.vocab.stoi[token] for token in tokens]
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.Encoder_LSTM(sentence_tensor)

    outputs = [german.vocab.stoi["<sos>"]]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)

        with torch.no_grad():
            output, hidden, cell = model.Decoder_LSTM(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()

        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == german.vocab.stoi["<eos>"]:
            break

    translated_sentence = [german.vocab.itos[idx] for idx in outputs]
    return translated_sentence[1:]

# 用來評估模型的函式: bleu
def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)

def checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss):
    print('saving')
    print()
    state = {'model': model,'best_loss': best_loss,'epoch': epoch,'rng_state': torch.get_rng_state(), 'optimizer': optimizer.state_dict(),}
    torch.save(state, './checkpoint-NMT')
    torch.save(model.state_dict(),'./checkpoint-NMT-SD')

In [25]:
epoch_loss = 0.0
# num_epochs = 100
num_epochs = 50
best_loss = 999999
best_epoch = -1
sentence1 = "a trendy girl talking on her cellphone while gliding slowly down the street ."
ts1  = []

for epoch in range(num_epochs):
  print("Epoch - {} / {}".format(epoch+1, num_epochs))
  model.eval()
  translated_sentence1 = translate_sentence(model, sentence1, german, english, device, max_length=50)
  print(f"Translated example sentence 1: \n {translated_sentence1}")
  ts1.append(translated_sentence1)

  model.train(True)
  for batch_idx, batch in enumerate(train_iterator):
    input = batch.src.to(device)
    target = batch.trg.to(device)

    # Pass the input and target for model's forward method
    output = model(input, target)
    output = output[1:].reshape(-1, output.shape[2])
    target = target[1:].reshape(-1)

    # Clear the accumulating gradients
    optimizer.zero_grad()

    # Calculate the loss value for every epoch
    loss = criterion(output, target)

    # Calculate the gradients for weights & biases using back-propagation
    loss.backward()

    # Clip the gradient value is it exceeds > 1
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    # Update the weights values using the gradients we calculated using bp 
    optimizer.step()
    step += 1
    epoch_loss += loss.item()
    writer.add_scalar("Training loss", loss, global_step=step)

  if epoch_loss < best_loss:
    best_loss = epoch_loss
    best_epoch = epoch
    checkpoint_and_save(model, best_loss, epoch, optimizer, epoch_loss) 
    if ((epoch - best_epoch) >= 10):
      print("no improvement in 10 epochs, break")
      break
  print("Epoch_Loss - {}".format(loss.item()))
  print()
  
print(epoch_loss / len(train_iterator))



Epoch - 1 / 50
Translated example sentence 1: 
 ['jüngeres', 'entgegen', 'gruppenfoto', 'gruppenfoto', 'übergibt', 'übergibt', 'rast', 'ziehen', 'lachen', 'lachen', 'turnschuhe', 'lachen', 'turnschuhe', 'stöcken', 'herzen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen', 'rostigen']
saving

Epoch_Loss - 4.623289108276367

Epoch - 2 / 50
Translated example sentence 1: 
 ['ein', 'mann', 'in', 'einem', 'einem', '<unk>', '<unk>', '<unk>', '<unk>', '.', '<eos>']
Epoch_Loss - 4.101475715637207

Epoch - 3 / 50
Translated example sentence 1: 
 ['ein', 'mann', 'in', 'einem', '<unk>', '<unk>', 'einem', '<unk>', 'auf', 'einem', '<unk>', '.', '